In [1]:
cd ..

/Users/ali/Documents/tomfoolery/oleorcarl


In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV 
from sklearn.utils.fixes import loguniform
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC



In [22]:
from src.database import Student
import numpy as np

students = list(Student.select())

X = np.stack([student.face for student in students if student.face is not None])

y = np.array([
    bool(student.school == "stolaf")
    for student in students
    if student.face is not None
])  # fmt: skip

print(X.shape, y.shape)


(862, 128) (862,)


In [50]:
# use grid search to find the best values for C and gamma

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.svm import SVC


# ranges from https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf (p. 6)

param_grid = [
    {
        "svc__kernel": ["linear"],
        "svc__C": np.logspace(-3, 6, 10, base=2),
    },
    {
        "svc__kernel": ["rbf"],
        "svc__C": np.logspace(-5, 15, 5, base=2),
        "svc__gamma": np.logspace(-15, 3, 5, base=2),
    },
    {
        "svc__kernel": ["poly"],
        "svc__C": np.logspace(-5, 15, 5, base=2),
        "svc__degree": [2, 3, 4],
        "svc__gamma": np.logspace(-15, 3, 5, base=2),
    },
]

model = GridSearchCV(
    estimator=make_pipeline(StandardScaler(), SVC()),
    param_grid=param_grid,
    scoring="accuracy",
    n_jobs=12,  # use all available threads
    refit=True,  # when done, fit the model with the best parameters found
    cv=5,  # 5-fold CV
)

model.fit(X, y)

print(model.best_params_, model.best_score_, model.refit_time_)

model


{'svc__C': 4.0, 'svc__kernel': 'linear'} 0.6670453017878748 1.4138727188110352


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             n_jobs=12,
             param_grid=[{'svc__C': array([ 0.125,  0.25 ,  0.5  ,  1.   ,  2.   ,  4.   ,  8.   , 16.   ,
       32.   , 64.   ]),
                          'svc__kernel': ['linear']},
                         {'svc__C': array([3.1250e-02, 1.0000e+00, 3.2000e+01, 1.0240e+03, 3.2768e+04]),
                          'svc__gamma': array([3.05175781e-05, 6.90533966e-04, 1.56250000e-02, 3.53553391e-01,
       8.00000000e+00]),
                          'svc__kernel': ['rbf']},
                         {'svc__C': array([3.1250e-02, 1.0000e+00, 3.2000e+01, 1.0240e+03, 3.2768e+04]),
                          'svc__degree': [2, 3, 4],
                          'svc__gamma': array([3.05175781e-05, 6.90533966e-04, 1.56250000e-02, 3.53553391e-01,
       8.00000000e+00]),
                          'svc__kernel': ['poly']}],
             scoring='accuracy')

In [72]:
import yaml, json
from pprint import pformat

def _write_yaml(object, filepath: str | None = None) -> None | str:
    object = json.loads(json.dumps(object))  # force yaml to work nicely
    try:
        if filepath is None:
            return yaml.safe_dump(object)

        with open(filepath, "wt", encoding="utf-8") as f:
            yaml.safe_dump(object, f)

    except Exception as e:
        print(f"Unable to save\n{pformat(object)}.")
        print(e.with_traceback())


print(_write_yaml(model.best_params_))

svc__C: 4.0
svc__kernel: linear



In [ ]:
from sklearn.model_selection import LeaveOneOut

# (hopefully do this in a thread pool of some sort)

# for every student in the database
    # train a classifier on everyone else
    # store the resulting score for that student

In [78]:
import pandas as pd

df = pd.DataFrame.from_dict(model.cv_results_).set_index("rank_test_score", drop=True).sort_index()
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__kernel,param_svc__gamma,param_svc__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
rank_test_score,,,,,,,,,,,,,,,,
1,1.426551,0.207035,0.005613,0.000679,4.0,linear,NaN,NaN,"{'svc__C': 4.0, 'svc__kernel': 'linear'}",0.687861,0.653179,0.680233,0.697674,0.616279,0.667045,0.029376
2,7.594706,1.501424,0.005497,0.001377,16.0,linear,NaN,NaN,"{'svc__C': 16.0, 'svc__kernel': 'linear'}",0.676301,0.641618,0.668605,0.709302,0.604651,0.660095,0.035133
3,0.114467,0.022005,0.005900,0.000445,0.25,linear,NaN,NaN,"{'svc__C': 0.25, 'svc__kernel': 'linear'}",0.676301,0.658960,0.639535,0.703488,0.622093,0.660075,0.028333
4,0.071156,0.004299,0.005490,0.000663,0.125,linear,NaN,NaN,"{'svc__C': 0.125, 'svc__kernel': 'linear'}",0.676301,0.641618,0.656977,0.703488,0.616279,0.658933,0.029698
4,2.817880,0.680249,0.006302,0.001364,8.0,linear,NaN,NaN,"{'svc__C': 8.0, 'svc__kernel': 'linear'}",0.676301,0.641618,0.668605,0.697674,0.610465,0.658933,0.030150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.057119,0.003380,0.030182,0.000710,32768.0,rbf,8.0,NaN,"{'svc__C': 32768.0, 'svc__gamma': 8.0, 'svc__k...",0.578035,0.578035,0.575581,0.575581,0.581395,0.577726,0.002138
69,0.038487,0.000996,0.008347,0.001078,0.03125,poly,0.000691,2,"{'svc__C': 0.03125, 'svc__degree': 2, 'svc__ga...",0.578035,0.578035,0.575581,0.575581,0.581395,0.577726,0.002138
69,0.053796,0.004048,0.027919,0.001230,1024.0,rbf,8.0,NaN,"{'svc__C': 1024.0, 'svc__gamma': 8.0, 'svc__ke...",0.578035,0.578035,0.575581,0.575581,0.581395,0.577726,0.002138


In [76]:
df.sort_values(["score"],)

KeyError: 'score'

In [1]:
cd ..

/Users/ali/Documents/tomfoolery/oleorcarl


In [20]:
import pandas as pd

from src.settings import GRID_SEARCH_RESULTS_PATH

df = pd.read_csv(GRID_SEARCH_RESULTS_PATH, index_col=0)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__C,param_clf__class_weight,param_clf__gamma,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
527,0.092567,0.018513,0.035893,0.013867,SVC(),1.533642,NaN,0.003402,"{'clf': SVC(), 'clf__C': 1.533641829168921, 'c...",0.728324,...,0.662791,0.604651,0.693642,0.658960,0.674419,0.651163,0.610465,0.660613,0.034117,1
661,0.255244,0.015657,0.065403,0.007984,SVC(),369.927210,NaN,0.000248,"{'clf': SVC(), 'clf__C': 369.9272095359148, 'c...",0.716763,...,0.668605,0.622093,0.658960,0.618497,0.656977,0.668605,0.633721,0.659501,0.028158,2
681,0.118681,0.019129,0.056547,0.005748,SVC(),0.997362,NaN,0.004187,"{'clf': SVC(), 'clf__C': 0.9973615583554495, '...",0.722543,...,0.651163,0.610465,0.693642,0.670520,0.627907,0.662791,0.627907,0.659447,0.031274,3
462,0.129575,0.011173,0.051923,0.010392,SVC(),2.820713,NaN,0.002393,"{'clf': SVC(), 'clf__C': 2.8207126641132336, '...",0.734104,...,0.645349,0.616279,0.676301,0.653179,0.680233,0.639535,0.616279,0.657723,0.032668,4
536,0.077285,0.022798,0.031961,0.013875,SVC(),37.494816,NaN,0.000604,"{'clf': SVC(), 'clf__C': 37.4948161688438, 'cl...",0.699422,...,0.662791,0.622093,0.676301,0.630058,0.668605,0.662791,0.639535,0.656597,0.021711,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,0.082427,0.012431,0.042881,0.008391,SVC(),0.136029,balanced,0.957603,"{'clf': SVC(), 'clf__C': 0.13602884198636284, ...",0.387283,...,0.436047,0.406977,0.427746,0.601156,0.540698,0.430233,0.395349,0.450665,0.064111,996
497,0.091090,0.012743,0.064328,0.010812,SVC(),0.048630,balanced,0.297991,"{'clf': SVC(), 'clf__C': 0.04863014889692455, ...",0.387283,...,0.436047,0.406977,0.427746,0.601156,0.540698,0.430233,0.395349,0.450665,0.064111,996
933,0.141366,0.011655,0.068259,0.009615,SVC(),0.019333,balanced,0.023479,"{'clf': SVC(), 'clf__C': 0.019332881529287756,...",0.387283,...,0.436047,0.406977,0.427746,0.398844,0.540698,0.430233,0.395349,0.442062,0.056456,998
2,0.119833,0.017284,0.092721,0.040678,SVC(),0.006619,balanced,0.004065,"{'clf': SVC(), 'clf__C': 0.0066187210646597655...",0.387283,...,0.563953,0.406977,0.427746,0.398844,0.459302,0.430233,0.395349,0.435085,0.048272,999


In [2]:
cd ..

/Users/ali/Documents/tomfoolery/oleorcarl


In [5]:
from sklearn.model_selection import validation_curve, RepeatedKFold, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC

import numpy as np

from src.classifier import get_vectors_and_labels

X, y = get_vectors_and_labels()
gamma_values = np.logspace(-5, 2, num=10)

clf = make_pipeline(StandardScaler(), SVC(C=10))

train_scores, test_scores = validation_curve(
    estimator=clf,
    X=X,
    y=y,
    param_name="svc__gamma",
    param_range=gamma_values,
    cv=KFold(n_splits=5, shuffle=True)
)


In [7]:
%matplotlib notebook

from matplotlib import pyplot as plt

plt.figure()
plt.axes()

# plt.plot(gamma_values, train_scores.mean(axis=1), "g.-")
plt.plot(gamma_values, test_scores.mean(axis=1), "r.-")
plt.xscale("log")
# print(gamma_values[test_scores.mean(axis=1).argsort()])
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

c_values = np.logspace()
gamma_values = np.logspace()

params = [
    {
        "svc__kernel": ["rbf"],
        "svc__gamma": gamma_values,
        "svc__c": c_values,
    },
    {
        "svc__kernel": ["linear"],
        "svc__c": c_values,
    }
]

pipeline = make_pipeline(StandardScaler(), SVC())

results = pd.DataFrame(
    GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        cv=RepeatedKFold(n_repeats=2, n_splits=5),
    ).cv_results_
)